# 소개

이 글과 코드는 
원문:https://colab.research.google.com/github/dream80/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab_V2.ipynb#scrollTo=5H85SsIW1PI3 을 한국어로 번역한 것입니다.

In [ ]:
#@title 할당 된 장치 보기
# Check the device, is it K80 or T4, if it is K80...！
! /opt/bin/nvidia-smi

# [1]첫번째 단계 workspace 준비하기// 다시 키면 다시 누르기/ 표준 런타임 변경
Google 드라이브의 작업공간 지정 디렉토리를 선정해 업로드하기.
지정 경로에 노트 업로드된 압축패키지를  workspace.zip으로 해야함
google cloud 디스크 주소：https://drive.google.com/drive/my-drive  

In [ ]:
#@title Mounting 시작하기 
#Mount Google Cloud Disk
#Click the link to authorize, copy the authorization code, fill in the frame, and then press Enter.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive
mkdir: cannot create directory ‘DeepFaceLab’: File exists
/content/drive/My Drive/DeepFaceLab


설치가 성공 후 공식 작업 공간을로드하거나 Google 클라우드 디스크를 통해 자신의 작업 공간으로 파일을 업로드 할 수 있습니다. 
지정된 경로에 노트는 압축 패키지형식으로 저장되어있습니다. 파일 이름은 workspace.zip해야합니다.

이 작업에는, DST SRC를 포함해야 하며, 내용에는 모델이 포함되어 있지 않으므로 

모델 파일은 모델 디렉토리에 직접 업로드 합니다.
v1.12.27 이후에는 faceset.pak를 정렬 된 디렉토리에 직접 업로드 할 수 있으며, 
압축을 풀지 않고도 빠르게 로드하여 사용할 수 있습니다.

In [ ]:
#@title [1]샘플 프로젝트 workspace 로드하기 
# 1. You can use this line to practice and directly git clone a workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

/content/drive/My Drive/DeepFaceLab
Cloning into 'workspace'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 38 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (38/38), done.


In [ ]:
#@title 프로젝트 workspace 압축해제(1.12.27v에서부터는 실행할 필요 없음)
#Need to upload workspace.zip to the Google Cloud Disk DeepFaceLab directory
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  workspace.zip

#2번째 단계: DeepFaceLab 설치하기
소스코드를 얻고 상황에 다라 버전을 선택
620: 안정
1019: SAEHD지원
마지막: 최신버전
 

In [ ]:
#@title [1]설치 시작

Version = "v1.12.27" #@param ["620", "v1.10.19","v1.11.09","v1.11.12","v1.12.27", "last"]
%cd /content/drive/My Drive/DeepFaceLab
!rm -fr DeepFaceLab_Colab
if Version=="620":
  print("620版加载中....")
  # Get DFL source code v1.6.1 stable version
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

/content/drive/My Drive/DeepFaceLab
v1.12.27加载中....
Cloning into 'DeepFaceLab_Colab'...
remote: Enumerating objects: 3955, done.
remote: Total 3955 (delta 0), reused 0 (delta 0), pack-reused 3955
Receiving objects: 100% (3955/3955), 785.54 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (2512/2512), done.
Note: checking out '02e7f1aaeee8a3f6e05220af3710318705dc50b4'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

Checking out files: 100% (253/253), done.
/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-8hkd2r49
  Running command git clone -q https:

     |████████████████████████████████| 12.4MB 246kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# 3번째 단계: 얼굴 추출하기

이 단계는 엄밀히 말해 두 번의 얼굴 추출 과정이 필요합니다. 
처음에는 사용할(target) 얼굴의 src를, 두 번째는 입힐 dst를 선택하여 각각 추출해야 합니다.

In [ ]:
#@title 추출 시작
target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  


/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Traceback (most recent call last):
  File "main.py", line 331, in <module>
    arguments.func(arguments)
  File "main.py", line 220, in process_videoed_extract_video
    from mainscripts import VideoEd
  File "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/mainscripts/VideoEd.py", line 3, in <module>
    import ffmpeg
ModuleNotFoundError: No module named 'ffmpeg'
WARNING !!! 
 /content/drive/My Drive/DeepFaceLab/workspace/data_src/aligned contains files! 
 They will be deleted. 
 Press enter to continue.

In [ ]:
#@title 정렬 시작
target = "src" #@param ["src","dst"]
order = "face" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #Src sort, you can view the results through Google Cloud Disk, delete bad pictures
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

python3: can't open file 'main.py': [Errno 2] No such file or directory


# 4단계: 모델 트레이닝


SAEHD, Quick96, H128, SAE, DF, LIAEF128 모델을 지원합니다. 
모델의 시작에서는 매개변수를 구성해야 하며, 기본 셋팅값은 Enter로 입력합니다.
학습하지 않으려면 중지를 클릭/ 단 예외가 발생하지만 다음 단계에서 계속 훈련됨으로 오류는 일어나지 않습니다.


In [ ]:
#@title 트레이닝 시작
Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR","Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
Running trainer.

/!\ Training data src directory does not exist.

Traceback (most recent call last):
  File "main.py", line 331, in <module>
  File "main.py", line 175, in process_train
    Trainer.main(args, device_args)
  File "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/mainscripts/Trainer.py", line 198, in main
    e.wait() #Wait for inital load to occur.
  File "/usr/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
KeyboardInterrupt


**학습중인 모델 미리보기：**
https://www.wxp123.me/p/329

요약: 마우스 왼쪽> 파일> /driver/…./model/미리보기.jpg 두 번 클릭 
  


# 5단계: 출력 변환

사진 변환시작 : 훈련에 선택했던 설정으로 사진의 변환 설정을 맞추어 실행합니다.
	이미지를 비디오로 /단계에서는 이 학습된 이미지들을 연결해 하나의 영상으로 취합합니다.
일례로 H128로 트레이닝했다면,출력도 H128로 설정해야 합니다.


In [ ]:
#@title 사진 변환 시작

Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [ ]:
#@title 이미지->비디오
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#6단계: Training 연달아 하기
두번째 훈련을 시작하거나 연결이 끊어진 후 모델트레이닝을 계속하면-> 단계를 모두 수행할 필요가 없습니다.

(1)	클라우드 디스크 마운트
(2)	셋팅설치
(3)	트레이닝 시작

**주의: ** workspace.zip은 src 와 dst만 포함,
만일 모델이 함께 저장되면 모델이 오버라이드 됩니다.



In [ ]:
#@title 한번의 클릭으로 실행하기
#Mount Google Cloud Disk
#Click the link to authorize, copy the authorization code, fill in the frame, and then press Enter.

Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Recover and unzip, improve loading speed, save a lot of time
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  -o workspace.zip


# Enter the DeepFaceLab_Colab directory
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# Install Python dependencies
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# Start training SAE. If it is another model, modify the following parameters, such as H128.
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

In [ ]:
#@title 한번의 클릭으로 실행하기>=1.12.27
#Mount Google Cloud Disk
#Click the link to authorize, copy the authorization code, fill in the frame, and then press Enter.

Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Enter the DeepFaceLab_Colab directory
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# Install Python dependencies
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# Start training SAE. If it is another model, modify the following parameters, such as H128.
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

# Tool

1.11.27 이전에는 데이터를 zip 파일로 수동 압축하여 로딩 속도를 높여야했습니다.
1.11.27 이후에는 아래와 같은 스크립트를 사용하여 패키지 및 압축 해제 할 수 있습니다.
압축 된 데이터로 모델을 훈련시키고 감압없이 출력을 변환 할 수 있으며, 이는 로딩 속도를 높입니다.

In [ ]:
#@title Material packaging

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [ ]:
#@title Material unpacking

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [ ]:
#@title Material enhancement

target = "src" #@param ["src"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  

# 출처

Google 클라우드 주소 : https://drive.google.com/drive/my-drive

작성자 이메일 : wpgdream@gmail.com

튜토리얼 사용 : https://www.wxp123.me

https://github.com/dream80/DeepFaceLab_Colab
